<a href="https://colab.research.google.com/github/alexis12895/TFG_practica/blob/main/Redes_neuronales/Generador_texto_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#SOLO SI QUIEREMOS importar ficheros desde GOOGLE COLAB de aqui, NO HACE FALTA EJECUTARLO
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#LIBRERIAS

#Usamos Numpy y ajustamos la semilla del generador aleatorio
import numpy as np
np.random.seed(5) #Nos permite replicar el entrenamiento en otro ordenador, garantiza que los parametros se inicializan de forma aleatoria pero con los mismos valores.

#Para leer los datos desde una web
import urllib.request

#Importamos de Keras Input,Dense y SimpleRNN, que nos dara una celda recuerrente
from keras.layers import Input, Dense, SimpleRNN

#importamos la estructura modelo y el optimizador Gradiente Descendente(SGD) para entrenar el modelo
from keras.models import Model
from tensorflow.keras.optimizers import SGD

#Para poder representar la entrada y la salida al modelo en el formato adecuado
from tensorflow.keras.utils import to_categorical
from keras import backend as K


In [ ]:
#1. LECTURA DE DATOS
''' SI USAMOS LA LECTURA DE GOOGLE DRIVE, si no  #leemos los datos
nombres = open('/content/sample_data/nombres_dinosaurios.txt','r').read()
nombres = nombres.lower()
print('nombres: ', nombres)
'''

#leemos los datos
nombres = urllib.request.urlopen('https://raw.githubusercontent.com/codificandobits/Generacion_de_nombres_con_Redes_Recurrentes/master/nombres_dinosaurios.txt')  #Leemos el dataset des github, una URL
nombres = nombres.read()  #leemos la peticion de antes
nombres = nombres.lower() #lo ponemos en miniscula
nombres = nombres.decode("utf-8") #si hicieramos un print(nombres) sin esta linea, nos diria con una b al principio que los datos estan bytes y no str y no nos permitira concatenar "print('prueba' + nombres)// Pero con esta linea le estamos decoficando y pasando de bytes a str y ya me dejara concatenar y tambien no me aparecera la 'b' al principio "
print('nombres: ' +  nombres)

#Con list convierte en una lista los nombres y con set de esa lista me quita los caracteres repetidos, unificando los duplicados y asi obtenemos los 27 caracteres del alfabeto en codigo ANSII, incluyendo el caracter salto de linea
alfabeto = list(set(nombres)) #Esto lo hacemos por que la red no admite caracteres ,SOLO NUMEROS
print(alfabeto)

tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print(tam_datos)
print(tam_alfabeto)

#para luego en el entrenamiento necesitaremos representar cada caracter en el formato one-hot, es decir un vector de 27 elementos donde solo uno de esos elemetos sera distinto de cero, que sera el caracter que estemos representado. PARA esto necesitaremos crear un diccionario para poder definir el vector one-hot y marcar el indice correcto(donde este el 1), y creamos este diccionario del siguiente modo
car_a_ind = {car:ind for ind, car in enumerate(sorted(alfabeto))}
print(car_a_ind) #'\n(97 en ANSII) : 0 ...'

#cuando la red ya haya sido entrenada nos dara numeros y crearemos otro diccionario para convertir los indices en caracteres
ind_a_car = {ind:car for ind, car in enumerate(sorted(alfabeto))}
print(ind_a_car)


nombres: aachenosaurus
aardonyx
abdallahsaurus
abelisaurus
abrictosaurus
abrosaurus
abydosaurus
acanthopholis
achelousaurus
acheroraptor
achillesaurus
achillobator
acristavus
acrocanthosaurus
acrotholus
actiosaurus
adamantisaurus
adasaurus
adelolophus
adeopapposaurus
aegyptosaurus
aeolosaurus
aepisaurus
aepyornithomimus
aerosteon
aetonyxafromimus
afrovenator
agathaumas
aggiosaurus
agilisaurus
agnosphitys
agrosaurus
agujaceratops
agustinia
ahshislepelta
airakoraptor
ajancingenia
ajkaceratops
alamosaurus
alaskacephale
albalophosaurus
albertaceratops
albertadromeus
albertavenator
albertonykus
albertosaurus
albinykus
albisaurus
alcovasaurus
alectrosaurus
aletopelta
algoasaurus
alioramus
aliwalia
allosaurus
almas
alnashetri
alocodon
altirhinus
altispinax
alvarezsaurus
alwalkeria
alxasaurus
amargasaurus
amargastegos
amargatitanis
amazonsaurus
ammosaurus
ampelosaurus
amphicoelias
amphicoelicaudia
amphisaurus
amtocephale
amtosaurus
amurosaurus
amygdalodon
anabisetia
anasazisaurus
anatosaurus
a

In [ ]:
#2. (MODELO)CREACION DE LA RED RECCURENTE EN KERAS
#Tendra dos entradas x(vector de 27 elem), at-1(vector de 25 elem) y dos salidas y(vector de 27 elem), at(vector de 25 elem)

#el modelo tiene dos elementos, una celda recuerrente(x, at-1) y una salida at // una capa softmax de salida(at) y la salida(y)

n_a = 25 #numero de unidades en la capa en la oculta
#creamos dos contenedores
entrada = Input(shape=(None, tam_alfabeto)) #None nos dice que tendremos entradas de tam varialbes(nombres de dinosario) y el tam de alfa(27)
a0 = Input(shape=(n_a)) #para el estado oculto, vector de 25 elem

#creamos la celda recurrente, celda de 25 neuronas, con una funcion de activacion tangente hiperbolica y return_state para que nos de el nuevo estado oculto actualizado
celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)

#Ahora creamos la capa sofmax, con 27 neuronas de salida(tam_alfa), y usara la fun de act sofmax
capa_salida = Dense(tam_alfabeto, activation='softmax')

#creamos el modelo,  instanciamos la celda recurrente con las dos entradas de antes y en 'salida' almacenamos la activacion que nos da la capa recurrente
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida = []
salida.append(capa_salida(hs))

#creamos el modelo indicandole que tendremos dos entradas y la salida correspondiente
modelo = Model([entrada, a0], salida)

#añadimos el optimizador, el gradiente descendente
opt = SGD(lr=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy')



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
#4. ENTRENAMIENTO RED RECURRENTE

''' SI LO HACES CON LA IMPORTACION DE GOOGLE COLAB
with open('/content/sample_data/nombres_dinosaurios.txt','r') as f:
  ejemplos = f.readlines()
'''

#abrimos y nos devolvera una lista con todos los nombre de los dinosarius, pero nos devuelve la lista en bytes
with urllib.request.urlopen('https://raw.githubusercontent.com/codificandobits/Generacion_de_nombres_con_Redes_Recurrentes/master/nombres_dinosaurios.txt') as f:
  ejemplos = f.readlines()

#ahora ejemplos sera una lista en bytes con los nombres de los dinosaurios y asi consigo que todos los elementos(nombres) de la lista sean de tipo str
ejemplos = [x.decode("utf-8") for x in ejemplos]
print('Lista de ejemplos: ',ejemplos)

ejemplos = [x.lower().strip() for x in ejemplos]  #con strip me elimina los caracteres iniciales y finales :  leon    // pues quita los espacios de delante y detras
print("Lista de ejemplos sin saltos de linea: ",ejemplos)
np.random.shuffle(ejemplos)

print('Lista de ejemplos mezclada: ',ejemplos) #Estan los nombres mezclados

#crea ejemplos(nombres de dino) de entrenamiento usando un generador
def train_generator():
  while True:

    #Tomar un ejemplo aleatorio, un nombre de dino aleatorio
    ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]
    #print('nombre aleatorio: ', ejemplo)
    #print('nombre aleatorio len: ', len(ejemplo))

    #Convertir el ejemplo(nombre de dino) a representación numerica
    X = [None] + [car_a_ind[c] for c in ejemplo]
    #print('nombre dino: ', X)
    #print('nombre dino len: ', len(X))

    #Crear "Y", resultado de desplazar "X" un caracter a la derecha
    Y = X[1:] + [car_a_ind['\n']] #estoy añadiendo a patir del elemento 1 del vector X(contiene el nombre del dinosario en numeros) un caracter mas(\n) que es el 10: 0, esto nos dice que el '\n' equivale al 0
    #print('nombre Y: ', Y)
    #print('nombre Y len: ', len(Y))

    #Representar "X" y "Y" en formato one-hot
    x = np.zeros((len(X),1,tam_alfabeto))
    onehot = to_categorical(X[1:], tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
    x[1:,:,:] = onehot  #one-hoy de x, es decir tendra el numero de filas que sea len(Y) y en cada fila todo ceros y solo un 1 donde sea igual al caracter que hemos definido en nuestro diccionario
    y = to_categorical(Y, tam_alfabeto).reshape(len(X),tam_alfabeto) #one-hot de y, el que tiene un desplazamiento a la derecha

    #Activacion inicial (matriz de ceros)
    a = np.zeros((len(X), n_a))

    yield [x, a], y





Lista de ejemplos:  ['Aachenosaurus\n', 'Aardonyx\n', 'Abdallahsaurus\n', 'Abelisaurus\n', 'Abrictosaurus\n', 'Abrosaurus\n', 'Abydosaurus\n', 'Acanthopholis\n', 'Achelousaurus\n', 'Acheroraptor\n', 'Achillesaurus\n', 'Achillobator\n', 'Acristavus\n', 'Acrocanthosaurus\n', 'Acrotholus\n', 'Actiosaurus\n', 'Adamantisaurus\n', 'Adasaurus\n', 'Adelolophus\n', 'Adeopapposaurus\n', 'Aegyptosaurus\n', 'Aeolosaurus\n', 'Aepisaurus\n', 'Aepyornithomimus\n', 'Aerosteon\n', 'AetonyxAfromimus\n', 'Afrovenator\n', 'Agathaumas\n', 'Aggiosaurus\n', 'Agilisaurus\n', 'Agnosphitys\n', 'Agrosaurus\n', 'Agujaceratops\n', 'Agustinia\n', 'Ahshislepelta\n', 'Airakoraptor\n', 'Ajancingenia\n', 'Ajkaceratops\n', 'Alamosaurus\n', 'Alaskacephale\n', 'Albalophosaurus\n', 'Albertaceratops\n', 'Albertadromeus\n', 'Albertavenator\n', 'Albertonykus\n', 'Albertosaurus\n', 'Albinykus\n', 'Albisaurus\n', 'Alcovasaurus\n', 'Alectrosaurus\n', 'Aletopelta\n', 'Algoasaurus\n', 'Alioramus\n', 'Aliwalia\n', 'Allosaurus\n', '

In [ ]:
#ENTRENAMIENTO
#RNN con un total de 1000 iterraciones y en cada iter pasaran 80 ejemplos de entrenamiento
BATCH_SIZE = 80
NITS = 10000

for j in range(NITS):
    historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    #imprimir la evolucion del entranmiento cada 1000 iteraciones
    if j%1000 == 0:  #Lo ponemos a 100 por que es muy lento el programa y asi podemos ver como si evoluciona, reduciendo el error en cada iter
      print('\nIteracion : %d, Eroor: %f' % (j, historia.history['loss'][0]) + '\n')




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys



Iteracion : 0, Eroor: 2.460044


Iteracion : 1000, Eroor: 2.316428


Iteracion : 2000, Eroor: 2.224341


Iteracion : 3000, Eroor: 2.217159


Iteracion : 4000, Eroor: 2.145551


Iteracion : 5000, Eroor: 2.185068


Iteracion : 6000, Eroor: 2.185675


Iteracion : 7000, Eroor: 2.245562


Iteracion : 8000, Eroor: 2.213736


Iteracion : 9000, Eroor: 2.171989



In [ ]:
#5. GENERACION DE NOMBRES USANDO EL MODELO

#para generarlos utilizamos la celda recurrente(RNN) y la capa de salida softmax ya entrnedadas.
def generar_nombre(modelo, car_a_num, tam_alfabeto,n_a):

  #Inicializar a e y a ceros
  x = np.zeros((1,1,tam_alfabeto,))
  a = np.zeros((1, n_a))
  #print(x)

  #Nombre genera donde guardarlo y caracteres
  nombre_generado = ''
  fin_linea = '\n'
  car = -1

  #Iteramos sobre el modelo y vamos generando letras hasta que llegue a fin_linea o el nombre tenga 50 caracteres
  contador = 0;
  while(car != fin_linea and contador != 50):

    #Generacmos la prediccion usando la celda recurrente(RNN)
    a, _ = celda_recurrente(K.constant(x), initial_state = K.constant(a))
    y = capa_salida(a)
    prediccion = K.eval(y)

    #Esogemos el elemento con la probabilidad mas alta
    ix = np.random.choice(list(range(tam_alfabeto)), p=prediccion.ravel())
    #print('ix: ', ix)

    #convertimos el elemnto seleecionado a caracter y lo añadimos al nombre
    car = ind_a_car[ix]
    #print('LETRAAA: ',car)
    nombre_generado += car
    #print('P.NOMBRE: ', nombre_generado)

    #creamos x_(t+1)->entrada a la celda siguiente = y_t, y a_t = a_(t-1)->nuevo estado oculto
    x = to_categorical(ix, tam_alfabeto).reshape(1,1, tam_alfabeto)
    a = K.eval(a)

    #Actualizar y continuar
    contador+= 1

    #agregar el '\n' al nombre en lugar esperar a tener 50
    if(contador == 50):
      nombre_generado += '\n'
      
  print(nombre_generado)

In [ ]:
#Problema de memoria, ya que los nombres de extension corto son mas reales y los de extension larga pierden su corta memoria y nos bastante ficticios
for i in range(100):
  generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)


vpmuaustouras

sas



taumuusauranepnushehis

ha

tiidophemodyauhabsaenngumesherysgos

a

pausauja

tixianeverapsausauellhusausausieorusamameraushusat

lviauseusaurakeps

telys

hecithoteendalausesausausaus

ruraustosaiapherset

ytelcur

apuans

n

douralauravausaunodostodg



laus



trusausptor

maus

nodatrurhgss

presausaurabecaeni

lkac

thushosegzesausaptusaus

plusa

turura

us

ocaus

taus

l

diberadi

us

imematosaugetorlausa

xcpauavacery

fusaus

weqchaus

ioshaus

psaushisagiahurusalecrishuasaushyresapsausanthyus


s

erapiesacourans

dopaeema

oviochatrhaytepresautaus

ifr

peodypthyus

ilosa

kquabs

iazecrepgusa

opohaus

rs

er



saugiphose

cha

ruras

pachisaeusausauskesaus

ororerylitholthushes

s

kleiboenditesausausaurabieyrovhglyhos

axfescokeliaus

ostius

maurus

rpthuras

hamateusas

ocerusaerpsausapoprasausphes

ora

enama

usaus

urathas

ge

ang

aterusatatvehchiankauhes

echaptocas

akaostausanmemraurausaonhusus

ges

os

li

rura

rs

ereperamausausausau